In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps

# Import API key
from api_keys import g_key # Output File (CSV)



In [2]:
#import csv and then create heatmap. Look up syuntax from heatnmap actviity.
#Use .loc function in order to filter out the df for ideal weather
#Create hotel_df and add column called "Hotel Name"
#Run google API calls and after I can refer googledat activity
#From there I can use that to get api calls for the hotel name in those cities
#Use the same heatmap and just add marker layer -- Look in the 3rd day actvoty folder


In [3]:
#Store filepath in a variable
city_data = "../cities.csv"

In [4]:
#Read data file woth pandas library
city_data_df = pd.read_csv(city_data, encoding="ISO-8859-1")

In [5]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [6]:
#Narrow down the DataFrame to find your personal ideal weather conditions for a vacation.
temp_ideal = city_data_df.loc[(city_data_df["Max Temp"]>=65)&(city_data_df["Max Temp"]<=80)]
wind_ideal = temp_ideal.loc[(temp_ideal["Wind Speed"]<=5)]
weather_ideal_df = wind_ideal.loc[(wind_ideal["Cloudiness"]==1)]
weather_ideal_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07


In [7]:
#Create variables for hotel locations and humidity
hotel_locations = weather_ideal_df[["Lat", "Lng"]]
humidity = weather_ideal_df["Humidity"].astype(float)

In [10]:
#Create variable for heatmap
heat_map = gmaps.figure(center = [0,0] ,zoom_level = 1)

#Create variable for heat layer
heat_layer = gmaps.heatmap_layer(hotel_locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
heat_map.add_layer(heat_layer)

#Display heatmap
heat_map


Figure(layout=FigureLayout(height='420px'))

In [11]:
#Create hotel_df and add column "Hotel Name"
#Add "Hotel Name" to created dataframe and rename "hotel_df"
# create hotel file
hotel_df = weather_ideal_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42,
1,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,


In [12]:
#Establish parameters for heatmap
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

In [13]:
#Create for statment to iterate through lat and long to search for hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Query API url
    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request
    hotel_search = requests.get(query_url, params=params)
    
    # convert to json
    hotel_search = hotel_search.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_df["results"][0]["name"]
    except (KeyError, IndexError):
        print("Skipping")
hotel_df

Skipping
Skipping


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42,
1,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
#Marker added to mark hotels
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

heat_map.add_layer(markers)

# Display Map
heat_map

Figure(layout=FigureLayout(height='420px'))